<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/HRandVentparam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [1]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [2]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [3]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


Authenticated


In [4]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000


In [5]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [6]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# Exploring data
Am trying to follow :
https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/omop/sql/ddl/bigquery.sql

Is it possible to approx SDNN even if the HR is only per hour?

In [7]:
%%bigquery test --project $PROJECT_ID
select person_id, measurement_date, stddev_samp(value_as_number) as sd
from measurement m
where measurement_concept_id in ( 21490872 --Heart rate.beat-to-beat by EKG
) and value_as_number > 0 and value_as_number < 300
group by person_id, measurement_date
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
test.describe()

,person_id,sd
count,100.0,94.000000
mean,63029.16,7.784612
std,2513.879487,4.230975
min,58924.0,0.000000
25%,60462.75,5.434138
50%,65353.0,7.278327
75%,65384.0,9.355011
max,65396.0,26.166144


In [15]:
%%bigquery ecmolist --project $PROJECT_ID
select m.measurement_concept_id, c.concept_name, m.measurement_source_value
, min(measurement_datetime ) as first_date
, max(measurement_datetime ) as last_date
, count(distinct person_id) as n_patients
, count(*) as entries
  from measurement m inner join concept c on c.concept_id = m.measurement_concept_id
  where measurement_concept_id IN (2000000101
  , 2000000099
  , 2000000102
  , 2000000103
  , 2000000104
  , 2000000105
  , 2000000100
  , 2000000419
  , 2000000400
)
group by m.measurement_concept_id, c.concept_name, m.measurement_source_value
order by count(*) desc
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
ecmolist

,measurement_concept_id,concept_name,measurement_source_value,first_date,last_date,n_patients,entries
0,2000000419,ecmo arterial temperature,ECMO Tart;R UMCA PERF TEMP ART,2010-06-03 20:32:00+00:00,2023-08-27 00:09:00+00:00,2374,440524
1,2000000102,ECMO pressure arterial,ECMO Part;R UMCA ECMO ARTERIELE DRUK,2010-01-01 10:32:00+00:00,2022-10-05 14:53:00+00:00,302,28582
2,2000000103,ECMO pressure venous,ECMO Pven;R UMCA ECMO VENEUZE DRUK,2010-01-01 10:32:00+00:00,2022-10-05 14:53:00+00:00,249,23323
3,2000000104,ECMO rounds per minute,ECMO Pomp rotaties (RPM);R UMCA ECMO POMP ROTA...,2010-01-01 00:53:00+00:00,2022-10-05 14:53:00+00:00,230,21346
4,2000000101,ECMO FiO2,ECMO FiO2;R UMCA ECMO FI02,2010-01-01 10:32:00+00:00,2022-10-05 14:53:00+00:00,213,20641
5,2000000419,ecmo arterial temperature,ECMO Tart;R UMCA ECMO TEMP ARTERIEEL,2010-01-02 05:59:00+00:00,2020-11-12 19:32:00+00:00,114,10601
6,2000000400,ecmo venous temperature,ECMO Tven;R UMCA PERF TEMP VEN,2016-12-30 12:04:00+00:00,2022-10-05 14:53:00+00:00,134,10004
7,2000000104,ECMO rounds per minute,ECMO Pomp rotaties ingesteld (RPM);R UMCA GENE...,2016-12-31 21:15:00+00:00,2022-10-05 14:53:00+00:00,96,9588
8,2000000099,ECMO arterial flow,ECMO bloedflow ingesteld (l/min);R UMCA GENERI...,2016-12-31 21:15:00+00:00,2022-10-05 14:53:00+00:00,96,9048
9,2000000104,ECMO rounds per minute,Toerental_Ecla,2010-01-01 01:45:00+00:00,2013-12-25 21:32:00+00:00,15,3049


In [60]:
%%bigquery test --project $PROJECT_ID
select o.condition_concept_id, c.concept_name, o.condition_source_value
, count(distinct person_id) as n_patients, count(*) as entries
from condition_occurrence o inner join concept c on c.concept_id = o.condition_type_concept_id
where condition_type_concept_id IN (32879)
group by o.condition_concept_id, c.concept_name, o.condition_source_value
order by count(*) desc
limit 100


Query is running:   0%|          |

Downloading:   0%|          |

In [61]:
test


,condition_concept_id,concept_name,condition_source_value,n_patients,entries
0,0,Registry,"CABG alone, coronary artery bypass grafting",5647,7394
1,0,Registry,Operatief Cardiovasculair,3260,7111
2,0,Registry,Post-operative cardiovascular,3114,6801
3,0,Registry,Operatief Cardiovasculair - Cardiovasculair,2102,4753
4,0,Registry,Diagnose anders,1921,4511
...,...,...,...,...,...
95,0,Registry,Seizures (primary-no structural brain disease),522,655
96,0,Registry,Subarachnoid hemorrhage/intracranial aneurysm,605,649
97,0,Registry,"Cardiovascular surgery, other",570,649
98,0,Registry,Non-operatief Gastro-Intestinaal - Gastro-inte...,124,643


# Vent data per hour

In [49]:
%%bigquery ventpat --project $PROJECT_ID
with rr as
(
  with br as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 1175625 then 'BRspont'
       when measurement_concept_id = 3007469 then 'BRset'
       when measurement_concept_id = 3012410 then 'TV'
       when measurement_concept_id = 3022875 then 'PEEP'
       when measurement_concept_id in ( 3025408, 2000000204) then 'FiO2'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
         1175625 -- Breath rate spontaneous
        , 3007469 -- Breath rate setting Ventilator
        , 3012410 -- Tidal volume setting Ventilator
        , 3022875 -- Positive end expiratory pressure setting Ventilator
        , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
        , 2000000204 -- fio2 setting
  )
  and value_as_number is not null
  and value_as_number > 0 and value_as_number < 200
  )
  select * from br
  PIVOT (max(value_as_number) for concept_name in ('BRspont', 'BRset', 'TV','PEEP', 'FiO2'))
),
hr as (
select person_id, measurement_datetime, value_as_number
from measurement m
where measurement_concept_id in (
 	21490872 --Heart rate.beat-to-beat by EKG
)
and person_id in (select distinct person_id from rr )
and value_as_number > 0 and value_as_number < 300
),
trach as (
  select person_id, measurement_datetime, value_as_number as trachsize
  from measurement m
  where measurement_concept_id in ( 36305611  -- Tracheostomy tube diameter
  )
),
ecmo as (
select person_id, measurement_datetime, value_as_number as ecmoTart
  from measurement m
  where measurement_concept_id IN (
   2000000419 -- ecmo arterial temperature, this has the most entries
  )
    and value_as_number < 100 -- there were implausible entries of 692 deg celcius
),

 d as (
  select person_id, death_datetime
  from death
  where person_id in (select person_id from rr)
 )
select hr.person_id, hr.measurement_datetime, hr.value_as_number as hr_ekg
, rr.BRspont, rr.BRset, rr.TV, rr.PEEP, rr.FiO2
, trach.trachsize, ecmo.ecmoTart, d.death_datetime
from rr right outer join hr
on rr.person_id = hr.person_id
and rr.measurement_datetime = hr.measurement_datetime
left outer join trach
on rr.person_id = trach.person_id
and rr.measurement_datetime = trach.measurement_datetime
left outer join ecmo
on rr.person_id = ecmo.person_id
and rr.measurement_datetime = ecmo.measurement_datetime
left outer join d
on rr.person_id = d.person_id
order by hr.person_id, hr.measurement_datetime


Query is running:   0%|          |

Downloading:   0%|          |

In [50]:
ventpat.describe()

,person_id,hr_ekg,BRspont,BRset,TV,PEEP,FiO2,trachsize,ecmoTart
count,9524984.0,9.524984e+06,1.258919e+06,4.561040e+06,66658.000000,6.059618e+06,7.411690e+06,36.000000,210043.000000
mean,32828.870266,8.082980e+01,2.079562e+01,1.696402e+01,97.734218,6.249547e+00,5.663091e+01,7.611111,85.569195
std,18962.56034,2.069797e+01,9.767418e+00,6.462886e+00,61.837420,2.308584e+00,2.258185e+01,0.644882,14.970978
min,3.0,1.000000e+00,1.000000e-01,1.000000e+00,0.150000,1.000000e-01,4.500000e-01,6.000000,6.900000
25%,16499.0,6.600000e+01,1.500000e+01,1.290000e+01,41.000000,5.000000e+00,4.000000e+01,7.000000,77.900000
50%,32794.0,7.900000e+01,2.070000e+01,1.500000e+01,95.000000,5.000000e+00,5.000000e+01,8.000000,92.500000
75%,49201.0,9.300000e+01,2.700000e+01,1.990000e+01,156.000000,8.000000e+00,7.000000e+01,8.000000,95.700000
max,65396.0,2.930000e+02,1.710000e+02,1.988000e+02,199.000000,1.280000e+02,1.040000e+02,8.000000,99.900000


In [51]:
ventpat[ventpat['hr_ekg']>200] # plausibility checks
# ventpat[ventpat['PEEP']>100] # plausibility checks
# ventpat[ventpat['ecmoTart']>90] # plausibility checks

,person_id,measurement_datetime,hr_ekg,BRspont,BRset,TV,PEEP,FiO2,trachsize,ecmoTart,death_datetime
14075,103,2016-12-31 22:37:00+00:00,232.0,NaN,12.0,NaN,4.0,100.0,NaN,NaN,NaT
14090,103,2016-12-31 22:52:00+00:00,221.0,NaN,12.0,NaN,5.0,100.0,NaN,NaN,NaT
52950,310,2016-12-31 19:13:00+00:00,218.0,NaN,NaN,NaN,0.3,63.0,NaN,78.1,NaT
75361,449,2009-12-31 15:12:00+00:00,208.0,NaN,NaN,NaN,4.0,80.0,NaN,NaN,NaT
88132,471,2016-12-31 17:15:00+00:00,214.0,NaN,NaN,NaN,5.0,57.0,NaN,NaN,2022-03-18 14:31:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
9511575,65336,2016-12-31 19:12:00+00:00,216.0,NaN,NaN,NaN,5.0,98.0,NaN,NaN,NaT
9516766,65357,2016-12-21 01:47:00+00:00,218.0,NaN,12.0,NaN,5.0,100.0,NaN,NaN,NaT
9516767,65357,2016-12-21 01:48:00+00:00,216.0,NaN,12.0,NaN,5.0,100.0,NaN,NaN,NaT
9516768,65357,2016-12-21 01:49:00+00:00,209.0,NaN,12.0,NaN,5.0,100.0,NaN,NaN,NaT


In [52]:
# ventpat[ventpat['person_id']==65357] # random patient check
# ventpat[ventpat['person_id']==31159] # random patient check
ventpat[ventpat['person_id']==690] # random patient check

,person_id,measurement_datetime,hr_ekg,BRspont,BRset,TV,PEEP,FiO2,trachsize,ecmoTart,death_datetime
122070,690,2016-12-31 13:52:00+00:00,62.0,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaT
122071,690,2016-12-31 13:54:00+00:00,163.0,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaT
122072,690,2016-12-31 13:55:00+00:00,174.0,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaT
122073,690,2016-12-31 13:57:00+00:00,60.0,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaT
122074,690,2016-12-31 13:58:00+00:00,58.0,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
122523,690,2017-01-12 08:05:00+00:00,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
122524,690,2017-01-13 07:35:00+00:00,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
122525,690,2017-01-14 00:05:00+00:00,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
122526,690,2017-01-14 08:35:00+00:00,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


# BGA, every lab

In [ ]:
%%bigquery bga --project $PROJECT_ID
with br as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 3010421 then 'pH'
     when measurement_concept_id = 3027315 then 'PaO2'
		 when measurement_concept_id = 3013290 then 'PaCO2'
		 when measurement_concept_id = 3006576 then 'HCO3'
		 when measurement_concept_id = 3012501 then 'BE'
		 when measurement_concept_id = 3047181 then 'Lactate'
     when measurement_concept_id = 3024928 then 'O2_venous'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
         3010421 -- pH of Blood
											,3027315 -- Oxygen [Partial pressure] in Blood
											,3013290 -- Carbon dioxide [Partial pressure] in Blood
											,3006576 -- Bicarbonate [Moles/volume] in Blood
											,3012501 -- Base excess in Blood by calculation
											,3047181 -- Lactate [Moles/volume] in Blood
                      ,3024928 -- Oxygen saturation in Venous blood
  )
  and value_as_number is not null
  )
  select * from br
  PIVOT (max(value_as_number) for concept_name in ('Lactate', 'pH', 'PaO2', 'PaCO2', 'HCO3','BE','O2_venous'))


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
bga.describe()

,person_id,Lactate,pH,PaO2,PaCO2,HCO3,BE,O2_venous
count,1624341.0,184323.000000,637678.000000,652377.000000,348036.000000,6.380370e+05,648109.000000,311047.000000
mean,32592.803046,2.095386,7.997381,45.639425,5.882058,2.851162e+01,2.895174,74.950771
std,18835.003566,5.991713,57.384907,42.981310,1.715303,2.503819e+03,4.666040,21.045521
min,1.0,-1.300000,0.000000,0.100000,0.900000,-2.410000e+01,-165.999986,0.000000
25%,16369.0,1.000000,7.350000,10.400000,4.800000,2.210000e+01,0.700000,74.000000
50%,32777.0,1.400000,7.410000,19.900000,5.500000,2.500000e+01,2.700000,79.000000
75%,48645.0,2.200000,7.450000,75.800000,6.500000,2.850000e+01,5.300000,85.000000
max,65396.0,2357.000001,7522.000000,741.000000,32.900000,1.999999e+06,283.299999,100.000000


# Here you can add your code to explore the data

The BGA times are different from the ventilator times. So a join is not possible. To be considered are 1) summarising by day to determine patterns? 2) standard deviations by day? by shift morning/day/night? Missing are demographics, reason for admission.   